In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfpd = tfp.distributions
import pandas as pd
import numpy as np
import pybullet as p
import time
import pybullet_data
import jsonpickle
import os
from grippers import Suction

######

opo = [((0.5000037315408677, -1.2578027465071083e-06, 0.0299896065223573),
  (2.4129193883630846e-05,
   -5.713909263958897e-06,
   -0.000139535226016604,
   0.9999999899575269)),
 ((0.5000171811001196, 0.030003322663749856, 0.02999161224668104),
  (5.547388841781149e-05,
   8.30373466679495e-05,
   0.00034631714928941396,
   0.9999999350459373)),
 ((0.5000253344706734, -0.029991060669803497, 0.029986860564352447),
  (-2.3977882126943463e-05,
   0.00012532323025734639,
   -0.0007254975149075948,
   0.9999997286862157)),
 ((0.5000100815127885, 2.3575947583770575e-05, 0.06497819323024515),
  (0.0002418671556515865,
   5.842083920951768e-05,
   0.70703536849138,
   0.7071781428957932)),
 ((0.5001145512921592, -0.015953101932637596, 0.0849436243432835),
  (0.00011145119591672905,
   0.0012030580413085596,
   -0.0002943632824824671,
   0.9999992267898201)),
 ((0.5002215631812987, 0.01605099968797359, 0.08495167915485884),
  (8.870674787543044e-05,
   0.0011656463937275553,
   -0.0005724952089380428,
   0.9999991528240578)),
 ((0.5002455506436819, 5.9477183719911714e-05, 0.11243244763682483),
  (0.7071638053257149,
   0.0003120534505993868,
   -0.0016628871956301317,
   0.7070477281386831)),
 ((0.50022835226991, 5.7957265197686254e-05, 0.13992064006586297),
  (9.014181157110362e-05,
   0.0013211041311034372,
   -0.0005026433204954867,
   0.9999989969535074)),
 ((0.5002106298758064, 2.60430624294914e-05, 0.1848885440206841),
  (5.617581528711806e-05,
   0.001354594470391037,
   -0.00028453428161927735,
   0.9999990404787102))]

######

ASSETS_ROOT = os.path.abspath("urdf")
obj_ids = {'fixed': [], 'rigid': [], 'deformable': []}
homej = np.array([-1, -0.5, 0.5, -0.5, -0.5, 0]) * np.pi

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(ASSETS_ROOT, 'plane/plane.urdf'))

origin = np.zeros(3)
noRotation = p.getQuaternionFromEuler([0,0,0])

ur5 = p.loadURDF(os.path.join(ASSETS_ROOT, 'ur5/ur5.urdf'), origin, noRotation, 
                useFixedBase=True)
suc = Suction(ASSETS_ROOT, ur5, 9, obj_ids)
n_joints = p.getNumJoints(ur5)
joints = [p.getJointInfo(ur5, i) for i in range(n_joints)]
joints = [j[0] for j in joints if j[2] == p.JOINT_REVOLUTE]

for j, hj in zip(joints, homej):
    p.resetJointState(ur5, j, hj)

# only static dimension shapes
shapes = ['cube', 'cuboid', 'thin_cuboid', 'thinner_cuboid']
valid_rots = {
    'cube': ([0], [0], [0]),
    'cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thin_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thinner_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
}

objPos = np.array([0.5,0,0.1])

def load_objr(otype, od=[0,0,0], rot=[0,0,0], tp=True):
    ret = p.loadURDF(os.path.join(ASSETS_ROOT, f'objects/{otype}.urdf'), objPos+np.array(od), rot)
    
    if tp:
        for _ in range(100):
            p.stepSimulation()
            time.sleep(1./240.)
        
    return ret

def load_obj2(otype, od=[0,0,0], rot=[0,0,0], tp=True):
    ret = p.loadURDF(os.path.join(ASSETS_ROOT, f'objects/{otype}.urdf'), np.array(od), rot)
        
    return ret

def load_objs(otlist, opo):
    for i, (ot, (pos, ori)) in enumerate(zip(otlist, opo)):
        pert = np.zeros(3)
        if i > 2:
            pert[:2] = np.random.uniform(-.007,0.007,2)
            print(pert)
            time.sleep(0.5)
        
        if i == 3:
            time.sleep(0.5)
        
        load_obj2(ot, pos+pert, ori)

# towards, right, up

# rots:
# rol head anti
# rol into/out of up/down
# rol into/out of left/right

objs = []
# objs.append(load_obj('cuboid', tp=False))
# objs.append(load_obj('cuboid', [0,0.03,0], tp=False))
# objs.append(load_obj('cuboid', [0,-0.03,0], tp=False))
# objs.append(load_obj('thin_cuboid', [0,0,0.1), [0,0,np.pi/2], tp=False))
# objs.append(load_obj('cube', [0,-0.016,0.1], tp=False))
# objs.append(load_obj('cube', [0,0.016,0.1]))
# objs.append(load_obj('cuboid', [0,0,0.1], [np.pi/2,0,0]))
# objs.append(load_obj('cube', [0,0,0.1]))
# objs.append(load_obj('cuboid', [0,0,0.2]))

time.sleep(5)
np.random.seed(40)
load_objs(['cuboid', 'cuboid', 'cuboid', 'thin_cuboid', 'cube', 'cube', 'cuboid', 'cube', 'cuboid'], opo)

# while True:
#     p.stepSimulation()
#     time.sleep(1./240.)
#     for i, ob in enumerate(objs):
#         print(f'o{i}: {p.getBasePositionAndOrientation(ob)}')
#     print()


# try:
#     opo
# except NameError:
#     opo = []
#     for i, ob in enumerate(objs):
#         opo.append(p.getBasePositionAndOrientation(ob))
#     print()

input()

p.disconnect()

pybullet build time: Jul 21 2022 19:48:53


[-0.00129238 -0.00622488  0.        ]
[ 0.00403949 -0.00297773  0.        ]
[-0.00069509 -0.00274523  0.        ]
[0.00036959 0.00173337 0.        ]
[0.00387486 0.00260738 0.        ]
[0.00673314 0.00141143 0.        ]
